In [ ]:
!pip install import_ipynb
!pip install tensorflow-datasets

In [1]:
 # .py로 매번 수정하기 귀찮아서 ipynb 파일로 import가능하게 만드는 라이브러리
import import_ipynb 

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import models
import resisc45_custom

import matplotlib.pyplot as plt

GoogLeNet = models.InceptionNet

# 제공하는 dataset 리스트 확인
#tfds.list_builders()[:-1]

importing Jupyter notebook from models.ipynb


In [7]:
#하이퍼 파라미터
batch_size = 128
num_epoch = 3
train_accuracies = []
test_accuracies = []


# 데이터 생성
# 아쉽게도 resisc45는 test set은 없다. minst는 test set을 별도로 준비하고 있다.
dataset, info = tfds.load('resisc45_custom', as_supervised = True, with_info = True)
dataset_train = dataset['train']


# print(dataset_train)
# print(info)

In [8]:
# 추가적으로 데이터 셋에 변동을 줘서, 학습효과를 늘려려고 하나, 꼭 필요하진 않다.
train_datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, zoom_range=0.1)


# GoogleNet 객체 생성
#
input_dim = (256, 256, 3)
output_dim = 45 # 총 45종류, 데이터셋에서 종류를 뽑아올 수는 없을까?
print(output_dim)

model = GoogLeNet(input_dim, output_dim)
model.build(input_shape = (None, 256, 256, 3))
model.summary()

45
Model: "inception_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          multiple                  1792      
                                                                 
 batch_normalization_17 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 activation_17 (Activation)  multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv2d_33 (Conv2D)          multiple                  9232      
                                                                 
 batch_normalization_18 (Bat  multiple          

In [9]:
# 입력값에 대해 원핫라벨링한 뒤, 교차 엔트로피에 통과하여 loss를 검사
loss_object = keras.losses.SparseCategoricalCrossentropy()
# 최적화는 adam
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name = 'train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
# test_loss = keras.metrics.Mean(name = 'test_loss')
# test_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')
    


#compile_by_users()

def train_step(image, label): # 훈련 이미지, 훈련 레이블
    with tf.GradientTape() as tape:
        
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
    
def test_step(image, label): # 훈련 이미지, 훈련 레이블
        
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)
    

In [10]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
for epoch in range(num_epoch):
    i = 0
    for image, label in dataset_train:
        #print(label)
        #print(image.shape)
        image = tf.expand_dims(image, axis=0)
        image = tf.cast(image, tf.float32)
        image /= 255
        
        label = tf.expand_dims(label, axis=0)
        
        for _image, _label in train_datagen.flow(image, label):
            printProgressBar(i + 1, len(dataset_train),  prefix = 'Progress:', suffix = 'Complete', length = 50)
            # Data Augmentabtion된 훈련 데이터를 _image, 훈련 레이블을 _label로 저장
            # Data Augmentabtion이란 데이터셋을 여러 방법으로 변경하여 데이터셋 개수를 늘리는 방법이다.
        # 회전을 시킨다거나, 임의의 범위를 잘라낸다건, 하나의 소르를 가지고 데이터를 우려내는 방법이다.
            train_step(_image, _label) # 모델에 data Augmentation된 훈련 데이터셋을 넣음
            i+=1
            break
    # 훈련 데이터 셋에 대한 학습 종료

#     for test_image, test_label in dataset_test:
#         test_step(test_image, test_label)
#         # 시험 데이터 셋 테스트 종료
        
    train_accuracies.append(train_accuracy.result())
    # test_accuracies.append(test_accuracy.result())
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100))
    #template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    # print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100,
    #                       test_loss.result(), test_accuracy.result() * 100))

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Epoch 1, Loss: 3.809382200241089, Accuracy: 2.019047498703003
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Epoch 2, Loss: 3.8091495037078857, Accuracy: 2.0063490867614746


In [ ]:
model.summary()

In [ ]:
!pip install pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다


In [ ]:
# 학습된 모델을 저장한다.
model.save('./googlenet_resisc45_custom_epoch_03/', include_optimizer=False)


# 오닉스 세이브를 위한 설치
!pip install -U tf2onnx
!python -m tf2onnx.convert --saved-model googlenet_resisc45_custom_epoch01 --output googlenet_resisc45_custom_epoch01.onnx

# 설치하고 나서 model_mnist.onnx를 이동한다.

In [ ]:
# 슬라이드로 보여주기

import sys
import numpy as np
import cv2
import glob

model = './googlenet/googlenet_resisc45_custom_epoch01'
config = './googlenet/deploy.prototxt'

# 모델을 불러온다.
net = cv2.dnn.readNet(model, config)

if net.empty():
    print('Network load fialed')
    sys.exit()
    
# 분류명 불러오기
classNames = []
with open('./googlenet/classification_classes.txt', 'r') as f:
    classNames = f.read().rstrip('\n').split('\n')

img_lists = glob.glob('./test/image*.jpg')

cv2.namedWindow('scene', cv2.WINDOW_NORMAL)
cv2.resizeWindow('scene', 488, 488) 

idx = 0
while True:
    if len(img_lists) <= 0:
        print('images is zeros')
        break
    
    print(idx)
    img = cv2.imread(img_lists[idx])
    if img is None:
        print('image read failed')
        break
        
        
    blob = cv2.dnn.blobFromImage(img, 1, (256, 256), (104, 117, 123), swapRB = False)
    net.setInput(blob)
    prob = net.forward()
    out = prob.flatten()
    classId = np.argmax(out)
    confidence = out[classId]
    category = classNames[classId]
    text = f'{category} ({confidence*100:4.2f} %)'
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX,
                0.5, (0, 0, 255), 1, cv2.LINE_AA)
    cv2.imshow('scene', img)

    if cv2.waitKey(3000) == 27:
        break

    idx +=1
    if idx >= len(img_lists):
        idx = 0
            
cv2.destroyAllWindows()